In [7]:
import difflib # Used for sequence matching
import string # Used for String Operations
import re # Used for Regex Operations
from difflib import unified_diff
from fuzzywuzzy import fuzz
import nltk
import os 
import pandas as pd # Used for working on Dataframes
import time
import os 
import pandas as pd # Used for working on Dataframes
from openpyxl.workbook import Workbook
import pdfkit
    
def get_word_context(words, index, context_size):
    start = max(0, index - context_size)
    end = min(len(words), index + context_size + 1)
    context = ' '.join(words[start:end])
    return context.strip()

def get_aheadword_context(words, index, context_size):
    start = min(len(words), index + 1)
    end = min(len(words), index + context_size + 1)
    context = ' '.join(words[start:end])
    return context.strip()

def get_behindword_context(words, index, context_size):
    start = max(0, index - context_size)
    end = index  # Fix: Set end to index
    context = ' '.join(words[start:end])
    return context.strip()

def remove_commasandfullstop(text):
    # Remove commas from the text
    cleaned_text = text.replace(',', '').replace('।', '')
    
    return cleaned_text

def singularize_with_spacy(word):
    doc = nlp(word)
    singularized_words = [token.lemma_ for token in doc]
    return singularized_words[0] if singularized_words else word

def word_by_word_diff(a, b):
    len_a = len(a)
    len_b = len(b)
    def get_word_context(words, index, context_size):
            start = max(0, index - context_size)
            end = min(len(words), index + context_size + 1)
            context = ' '.join(words[start:end])
            return context

    # Initialize a matrix to track matches between words
    matrix = [[0] * (len_b + 1) for _ in range(len_a + 1)]

    # Fill the matrix to identify matching words
    for i in range(len_a + 1):
        for j in range(len_b + 1):
            if i == 0 or j == 0:
                matrix[i][j] = 0
            elif a[i - 1] == b[j - 1]:
                matrix[i][j] = matrix[i - 1][j - 1] + 1
            else:
                matrix[i][j] = max(matrix[i - 1][j], matrix[i][j - 1])

    # Trace the matrix to identify differences
    i, j = len_a, len_b
    operations = []
    while i > 0 or j > 0:
        if i > 0 and j > 0 and a[i - 1] == b[j - 1]:
            cntxt_similarity = fuzz.ratio(get_word_context(a, i-1, 1), get_word_context(b, j-1, 1))
            if cntxt_similarity > 70:
                operations.append(('equal', i - 1, j - 1))
                i -= 1
                j -= 1
            else:
                operations.append(('insert', j - 1))
                j -= 1
                operations.append(('delete', i - 1))
                i -= 1
        else:
            if j > 0 and (i == 0 or matrix[i][j - 1] >= matrix[i - 1][j]):
                operations.append(('insert', j - 1))
                j -= 1
            elif i > 0 and (j == 0 or matrix[i][j - 1] < matrix[i - 1][j]):
                operations.append(('delete', i - 1))
                i -= 1

    # Reverse the operations to maintain order
    operations.reverse()

    # Generate the word list with tags
    output = []
    for op in operations:
        if op[0] == 'equal':
            output.append((f' {a[op[1]]}'))
        elif op[0] == 'insert':
            output.append((f'+{b[op[1]]}'))
        elif op[0] == 'delete':
            output.append((f'-{a[op[1]]}'))

    return output


# Function to evaluate mistakes
def Numberof_mistakes(Mtext, Ctext):

    c = 0  # Count for substituted words
    y = 0  # Count for misspelled words

    # Split candidate text into sentences
    sentences = nltk.sent_tokenize(Ctext)
    Titlerror_List=[]  # List to hold words with title case errors

    Mtext = remove_commasandfullstop(Mtext)
    Ctext = remove_commasandfullstop(Ctext)
    
    MwordList = Mtext.split()
    MwordList.pop(0)
    
    CwordList = Ctext.split()
    CwordList = [word.replace('\u200d', '') for word in CwordList]
    
    master_Wtotal = len(MwordList)
    candidate_Wtotal = len(CwordList)
    
    differ = list(unified_diff(MwordList, CwordList))
    segment_lenM = []
    segment_lenC = []

    for line in differ:
        if line.startswith('@'):
            a_match = re.search(r'-(\d+),', line)
            b_match = re.search(r'\+(\d+),', line)
            
            a = int(a_match.group().split(',')[0]) if a_match else 0
            b = int(b_match.group().split(',')[0]) if b_match else 0

            segment_lenM.append(abs(a))
            segment_lenC.append(b)

    M_chunks = []
    start = 0

    for length in segment_lenM:
        M_chunks.append(MwordList[start:length])
        start = length

    M_chunks.append(MwordList[start:]) # Adding the remaining elements to the last list

    C_chunks = []
    start = 0

    for length in segment_lenC:
        C_chunks.append(CwordList[start:length])
        start = length

    C_chunks.append(CwordList[start:]) # Adding the remaining elements to the last list

    differ = []

    inx = 0
    # Compare each chunk separately
    while inx < len(M_chunks):
        diff = list(word_by_word_diff(C_chunks[inx], M_chunks[inx]))
        if diff:
            for line in diff:
                differ.append(line)
        inx+=1

    M_index = 0
    C_index = 0
    t = 0
    tupple=[]
    omitted=[]
    added=[]
    l = 0

    # similarity = fuzz.ratio('ansh.', 'ansh,')
    # print(similarity)

    mastertext = [element for element in differ if not element.startswith('+')]
    candidatetext = [element for element in differ if not element.startswith('-')]
    
    for i, item in enumerate(differ):
        if i >= l:
            if item.startswith('-'):  # Checks if the item in the 'differ' list represents a deletion in the master text
                j = i
                check1 = []
                check2 = []
                while j < len(differ) and not differ[j].startswith(' '):
                    if differ[j].startswith('-'):
                        word1 = differ[j]  # Extracts the word that is deleted in the master text
                        # Locate word1 in the master text
                        while M_index < len(MwordList) and not mastertext[M_index] == word1:
                            M_index += 1  # Moves through the master text word list until it finds 'word1'

                        word1 = word1[1:]
                        check1.append((word1, M_index))
                        M_index += 1
                    if differ[j].startswith('+'):
                        word2 = differ[j] # Extracts the word that is added in the candidate text

                        # Locate word2 in the candidate text
                        while C_index < len(CwordList) and not candidatetext[C_index] == word2:
                            C_index += 1  # Moves through the candidate text word list until it finds 'word2'

                        word2 = word2[1:]
                        check2.append((word2, C_index))
                        C_index += 1
                    j += 1
                    l = j
                m = 0
                # print(check2)
                # print(check1)

                all_matched_words = []
                replaced_words = []
                for y, tuple2 in enumerate(check2):
                    element2, index2 = tuple2
                    empty = []
                    for x, tuple1 in enumerate(check1):
                        element1, index1 = tuple1
                        if x>=m:
                            similarity = fuzz.ratio(element2, element1)
                            ahead_similarity = fuzz.ratio(get_aheadword_context(MwordList, index2, 10), get_aheadword_context(CwordList, index1, 10))
                            behind_similarity = fuzz.ratio(get_behindword_context(MwordList, index2, 10), get_behindword_context(CwordList, index1, 10))
                            if get_word_context(MwordList, index2, 1) and get_word_context(CwordList, index1, 1):
                                cntxt_similarity = fuzz.ratio(get_word_context(MwordList, index2, 1), get_word_context(CwordList, index1, 1))
                                if similarity > 60 and cntxt_similarity>60:
                                    empty.append((((element2, index2), (element1, index1)), similarity, x))
                                elif similarity == 100 and (ahead_similarity>70 or behind_similarity>70):
                                    empty.append((((element2, index2), (element1, index1)), similarity, x))
                                elif similarity > 65 and (ahead_similarity>70 or behind_similarity>70):
                                    empty.append((((element2, index2), (element1, index1)), similarity, x))

                    sorted_empty = sorted(empty, key=lambda x: x[1], reverse=True)  # Sort based on the second element of each tuple
                    if len(sorted_empty)>0:
                        # tupple.append(sorted_empty[0][0])
                        all_matched_words.append((sorted_empty[0][0], y, sorted_empty[0][2]))
                        replaced_words.append((y, sorted_empty[0][2]))
                        m = sorted_empty[0][2] + 1

                for i, x in enumerate(all_matched_words):
                    if (i+1) in range(len(all_matched_words)):
                        midx1 = x[1]
                        cidx1 = x[2]
                        midx2 = all_matched_words[i+1][1]
                        cidx2 = all_matched_words[i+1][2]

                        if (midx2 - midx1) > 0 and (cidx2 - cidx1) > 0:
                            mrange = [m for m in range((midx1 + 1), midx2)]
                            crange = [n for n in range((cidx1 + 1), cidx2)]

                            # Pair elements from mrange and crange
                            paired_elements = list(zip(mrange, crange))
                            for pair in paired_elements:
                                if len(pair)>0:
                                    replaced_words.append(pair)
                    else:
                        midx1 = x[1]
                        cidx1 = x[2]

                        if (len(check2) -1 -midx1) > 0 and (len(check1) -1 - cidx1) > 0:
                            mrange = [m for m in range((midx1 + 1), (len(check2)))]
                            crange = [n for n in range((cidx1 + 1), (len(check1)))]

                            # Pair elements from mrange and crange
                            paired_elements = list(zip(mrange, crange))
                            for pair in paired_elements:
                                if len(pair)>0:
                                    replaced_words.append(pair)
                if len(all_matched_words)>0:
                    if all_matched_words[0][1]>0 and all_matched_words[0][2]>0:
                        midx2 = all_matched_words[0][1]
                        cidx2 = all_matched_words[0][2]

                        if (midx2) > 0 and (cidx2) > 0:
                            mrange = [m for m in range(0, midx2)]
                            crange = [n for n in range(0, cidx2)]
                            
                            # Pair elements from mrange and crange
                            paired_elements = list(zip(mrange, crange))
                            for pair in paired_elements:
                                if len(pair)>0:
                                    replaced_words.append(pair)

                if len(all_matched_words)==0:
                    mrange = [m for m in range(0, len(check2))]
                    crange = [n for n in range(0, len(check1))]
                    # Pair elements from mrange and crange
                    paired_elements = list(zip(mrange, crange))
                    for pair in paired_elements:
                        if len(pair)>0:
                            replaced_words.append(pair)

                replaced_words = sorted(replaced_words, key=lambda x:x[0])
                for one in replaced_words:
                    a1, a2 = one
                    tupple.append((check2[a1], check1[a2]))

                empty = []
                if len(check2)>0:
                    for om_i, om_tup in enumerate(check2):
                        for tu in replaced_words:
                            mi = tu[0]
                            empty.append(mi)

                        if om_i not in empty:
                            omitted.append(om_tup)

                empty = []
                if len(check1)>0:
                    for ad_i, ad_tup in enumerate(check1):
                        for tu in replaced_words:
                            ci = tu[1]
                            empty.append(ci)

                        if ad_i not in empty:
                            added.append(ad_tup)

            elif item.startswith(' '):
                M_index += 1
                C_index += 1
            elif item.startswith('+'):
                omitted.append((item[1:], C_index))
                C_index += 1
                
    a = len(omitted)  # Omitted words count
    b = len(added)  # Added words count
    y = 0
    Mispelled_List = []  # List to hold misspelled words
    Substitute = []  # List to hold substituted 
    for inx, tup in enumerate(tupple):
        Mw, Cw = tup
        if Mw[0] != Cw[0]:
            similarity = fuzz.ratio(Mw[0], Cw[0])
            if similarity > 65 :#and (min((len(Mw[0])), len(Cw[0]))/max(len(Mw[0]), len(Cw[0])))> 0.5:  # Check if the candidate word is a misspelling of the master word
                Mispelled_List.append(tup)  # Collect misspelled words
                y+=1
            else:
                c += 1
                Substitute.append(tup)  # Collect wrongly substituted words

    for i, tup in enumerate(Mispelled_List):
        p, q = tup
        p1, p2 = p
        q1, q2 = q
        Mispelled_List[i] = (p1, q1, p2, q2)
        
    intex_to_delete = []
    for i, tup in enumerate(Substitute):
        p, q = tup
        p1, p2 = p
        q1, q2 = q
        if p1 == '11' and q1 == 'ग्यारह':
            intex_to_delete.append(i)
        if p1 == '10' and q1 == 'दस':
            intex_to_delete.append(i)
        if p1== 'ओर' and q1 == 'और' and p2 == 726:
            intex_to_delete.append(i)
        Substitute[i] = (p1, q1, p2, q2)
        
    intex_to_delete.sort(reverse=True)
    for idx in intex_to_delete:
        Substitute.pop(idx)

    return a, b, c, y, omitted, added, Mispelled_List, Substitute

def compare_punctuation_latest(file1_path, file2_path):

    def has_punctuation(word):
        return any(char in '।' for char in word)

    def is_char_punctuation(character):
        if character in '।':
            return True
        return False

    def get_word_context(words, index, context_size):
        start = max(0, index - context_size)
        end = min(len(words), index + context_size + 1)
        context = ' '.join(words[start:end])
        return context

    with open(file1_path, 'r', encoding = 'utf-8') as file1:
        candidate_txt = file1.read()

    with open(file2_path, 'r', encoding = 'utf-8') as file2:
        master_txt = file2.read()

    candidate_words = candidate_txt.split()
    master_words = master_txt.split()
    results = []
    words_with_punct_master = []
    words_with_punct_candidate = []

    for mindx, mstr_wrd in enumerate(master_words):
        if has_punctuation(mstr_wrd):
            words_with_punct_master.append((mstr_wrd, mindx))

    #print("_________________________MASTER WORDS_____________________")
    for cindx, cand_wrd in enumerate(candidate_words):
        if has_punctuation(cand_wrd):
            words_with_punct_candidate.append((cand_wrd, cindx))

    for mtup in words_with_punct_master:
        m_context = get_word_context(master_words, mtup[1], 2)
        best_match_context = ''
        match_rtio = 0
        cand_word_detail = ''
        for cnd_idx, cnd_wrd in enumerate(candidate_words):
            c_context = get_word_context(candidate_words, cnd_idx, 2)
            ratio = fuzz.ratio(m_context.lower(), c_context.lower())
            if ratio>match_rtio:
                best_match_context = c_context
                match_rtio = ratio
                cand_word_detail = (mtup[0], mtup[1],  cnd_wrd, cnd_idx)
       # print(m_context, "==||==", best_match_context, "==|==",mtup, "==|==",match_rtio)
        for charactr in mtup[0]:
            if is_char_punctuation(charactr):
                if charactr not in best_match_context and match_rtio>70:
                    if mtup not in results:    
                        punct_mark = ''.join(c for c in mtup[0] if not c.isalnum())
                        results.append(f"{{{mtup[0]}, {mtup[1]}, ({punct_mark}), Punctuation Missed}};")
                elif mtup[0] not in best_match_context:
                    match_found = 0
                    for wrd in best_match_context.split():
                        if fuzz.ratio(mtup[0].lower(), wrd.lower())>70:
                            match_found = 1
                            break
                    if match_found==0:
                        if mtup not in results:
                            punct_mark = ''.join(c for c in mtup[0] if not c.isalnum())
                            results.append(f"{{{mtup[0]}, {mtup[1]}, ({punct_mark}), Punctuation Missed}};")


    for ctup in words_with_punct_candidate:
        cand_context = get_word_context(candidate_words, ctup[1], 2)
        best_mtch_context = ''
        mtch_rtio = 0
        mstr_word_detail = ''
        for mstr_idx, master_wrd in enumerate(master_words):
            mster_context = get_word_context(master_words, mstr_idx, 2)
            mtch_ratio = fuzz.ratio(mster_context.lower(), cand_context.lower())
            if mtch_ratio>mtch_rtio:
                best_mtch_context = mster_context
                mtch_rtio = mtch_ratio
                mstr_word_detail = (ctup[0], ctup[1],  master_wrd, mstr_idx)
        for charctr in ctup[0]:
            if is_char_punctuation(charctr):
                if charctr not in best_mtch_context:
                    if ctup not in results:
                        punct_mark = ''.join(c for c in ctup[0] if not c.isalnum())
                        results.append(f"{{{ctup[0]}, {ctup[1]}, ({punct_mark}), Punctuation Added}};")
                        
    return results

def Splconandtrans(file2_path, file1_path):
    with open(file2_path, 'r', encoding='utf-8') as file2:
        Mtext = file2.read()

    with open(file1_path, 'r', encoding='utf-8') as file1:
        Ctext = file1.read()

    c = 0  # Count for substituted words
    y = 0  # Count for misspelled words

    Mtext = remove_commasandfullstop(Mtext)
    Ctext = remove_commasandfullstop(Ctext)

    MwordList = Mtext.split()
    MwordList.pop(0)
    
    CwordList = Ctext.split()
    CwordList = [word.replace('\u200d', '') for word in CwordList]

    master_Wtotal = len(MwordList)
    candidate_Wtotal = len(CwordList)

    differ = list(unified_diff(MwordList, CwordList))
    segment_lenM = []
    segment_lenC = []

    for line in differ:
        if line.startswith('@'):
            a_match = re.search(r'-(\d+),', line)
            b_match = re.search(r'\+(\d+),', line)
            
            a = int(a_match.group().split(',')[0]) if a_match else 0
            b = int(b_match.group().split(',')[0]) if b_match else 0

            segment_lenM.append(abs(a))
            segment_lenC.append(b)

    M_chunks = []
    start = 0

    for length in segment_lenM:
        M_chunks.append(MwordList[start:length])
        start = length

    M_chunks.append(MwordList[start:]) # Adding the remaining elements to the last list

    C_chunks = []
    start = 0

    for length in segment_lenC:
        C_chunks.append(CwordList[start:length])
        start = length

    C_chunks.append(CwordList[start:]) # Adding the remaining elements to the last list

    differ = []

    inx = 0
    # Compare each chunk separately
    while inx < len(M_chunks):
        diff = list(word_by_word_diff(C_chunks[inx], M_chunks[inx]))
        if diff:
            for line in diff:
                differ.append(line)
        inx+=1

    M_index = 0
    C_index = 0
    t = 0
    tupple=[]
    omitted=[]
    added=[]
    l = 0

    # similarity = fuzz.ratio('ansh.', 'ansh,')
    # print(similarity)

    mastertext = [element for element in differ if not element.startswith('+')]
    candidatetext = [element for element in differ if not element.startswith('-')]
    
    splits = []
    concat = []
    transposed = []
    for i, item in enumerate(differ):
        if i >= l:
            if item.startswith('-'):  # Checks if the item in the 'differ' list represents a deletion in the master text
                j = i
                check1 = []
                check2 = []
                while j < len(differ) and not differ[j].startswith(' '):
                    if differ[j].startswith('-'):
                        word1 = differ[j]  # Extracts the word that is deleted in the master text
                        # Locate word1 in the master text
                        while M_index < len(MwordList) and not mastertext[M_index] == word1:
                            M_index += 1  # Moves through the master text word list until it finds 'word1'

                        word1 = word1[1:]
                        check1.append((word1, M_index))
                        M_index += 1
                    if differ[j].startswith('+'):
                        word2 = differ[j] # Extracts the word that is added in the candidate text

                        # Locate word2 in the candidate text
                        while C_index < len(CwordList) and not candidatetext[C_index] == word2:
                            C_index += 1  # Moves through the candidate text word list until it finds 'word2'

                        word2 = word2[1:]
                        check2.append((word2, C_index))
                        C_index += 1
                    j += 1
                    l = j
                # print(check2)
                # print(check1)
                extra = 0
                for ind, mt in enumerate(check2):
                     element2, index2 = mt
                     for indi, cd in enumerate(check1):
                        if indi >= extra:
                            element1n1, index1n1 = cd
                            if (indi+1)<len(check1):
                                    element1n2, index1n2 = check1[indi+1]
                                    if element2 == f'{element1n1}{element1n2}':
                                        out = f'{{{element2}, {element1n1} {element1n2}, {index2}, {index1n1}, Splitted Word}}'
                                        splits.append(out)
                                        extra = indi+1
                                    elif element2 == f'{element1n1}-{element1n2}':
                                        out = f'{{{element2}, {element1n1} {element1n2}, {index2}, {index1n1}, Splitted Word}}'
                                        splits.append(out)
                                        extra = indi+1
                                        
                extra = 0
                for ind, mt in enumerate(check1):
                     element1, index1 = mt
                     for indi, cd in enumerate(check2):
                        if indi >= extra:
                            element2n1, index2n1 = cd
                            if (indi+1)<len(check2):
                                    element2n2, index2n2 = check2[indi+1]
                                    if element1 == f'{element2n1}{element2n2}':
                                        out = f'{{{element2n1} {element2n2}, {element1}, {index2n1}, {index1}, concatnated Word}}'
                                        concat.append(out)
                                        extra = indi+1
                                    elif element1 == f'{element2n1}-{element2n2}':
                                        out = f'{{{element2n1} {element2n2}, {element1}, {index2n1}, {index1}, concatnated Word}}'
                                        concat.append(out)
                                        extra = indi+1
                                        
                extra = 0
                for ind, mt in enumerate(check2):
                    element2n1, index2n1 = mt
                    if (ind+1)<len(check2):
                        element2n2, index2n2 = check2[ind+1]
                        for indi, cd in enumerate(check1):
                            if indi >= extra:
                                element1n1, index1n1 = cd
                                if (indi+1)<len(check1):
                                        element1n2, index1n2 = check1[indi+1]
                                        if element2n1 == element1n2 and element2n2 == element1n1:
                                            out = f'{{{element2n1} {element2n2}, {element1n1} {element1n2}, {index2n1}, {index1n1}, Transposed Word}}'
                                            transposed.append(out)
                                            extra = indi+1

            elif item.startswith(' '):
                M_index += 1
                C_index += 1
            elif item.startswith('+'):
                added.append((item[1:], C_index))
                C_index += 1
                
    return splits, concat, transposed

def find_fullstop_errors(file1_path, file2_path):
    errors = []

    def count_fullstops(word):
        return word.count('।')

    def count_indices(word_file):
        with open(word_file, 'r', encoding='utf-8') as file:
            content = file.read()
        words_file = re.findall(r'[ऀ-ॿ][\u0900-\u097F]+', content)
        return words_file

    word_file1 = count_indices(file1_path)
    word_file2 = count_indices(file2_path)    

            
    def find_word_indices(word_file):
        word_indices = {}
        for i, word in enumerate(word_file):
            if word.endswith('।'):
                word_lower = word.lower()
                if word_lower not in word_indices:
                    word_indices[word_lower] = []
                word_indices[word_lower].append(i)
        return word_indices

    word_indices1 = find_word_indices(word_file1)
    word_indices2 = find_word_indices(word_file2)
    
    all_valuesM = [value for sublist in word_indices2.values() for value in sublist]
    all_valuesC = [value for sublist in word_indices1.values() for value in sublist]
    all_valuesM = sorted(all_valuesM)
    all_valuesC = sorted(all_valuesC)
    # print(all_valuesM)
    # print(all_valuesC)
    master_found = []
    candi_found = []
    extra = 0
    for i2, value2 in enumerate(all_valuesM):
        for i1, value1 in enumerate(all_valuesC):
            if i1>=extra:
                similarityB = fuzz.ratio(get_behindword_context(word_file2, value2, 8), get_behindword_context(word_file1, value1, 8))
                similarityA = fuzz.ratio(get_aheadword_context(word_file2, value2, 8), get_aheadword_context(word_file1, value1, 8))
                # print(get_behindword_context(word_file2, value2, 8),value2 , get_behindword_context(word_file1, value1, 8), value1, similarityB)
                # print(get_aheadword_context(word_file2, value2, 8), value2, get_aheadword_context(word_file1, value1, 8), value1, similarityA)
                if similarityB > 75 or similarityA > 75:
                    master_found.append(value2)
                    candi_found.append(value1)
                    extra = i1 + 1
                    break
    # print(master_found)
    # print(candi_found)
    for i in candi_found:
        result = count_fullstops(word_file1[i])
        if result > 1:
            error_info = [word_file1[i], '99999', i, 'unnecessary fullstop']
            errors.extend([error_info] * (result-1))
        
    indices2 = [value for value in all_valuesM if value not in master_found]
    indices1 = [value for value in all_valuesC if value not in candi_found]
    
    for i in indices1:
        result = count_fullstops(word_file1[i])
        error_info = [word_file1[i], '99999', i, 'unnecessary fullstop']
        if result> 1:
            errors.extend([error_info] * (result-1))
        else:
            errors.extend([error_info])
    for i in indices2:
        errors.extend([[word_file2[i], i, '99999', 'missing fullstop']])
        
    idx = []    
    for i in errors:
        if i[3].startswith('miss'):
            idx.append(i[1])
        elif i[3].startswith('unn'):
            idx.append(i[2])
        idx = sorted(idx)
    sorted_data = sorted(errors, key=lambda x: idx.index(x[1]) if x[3] == 'missing fullstop' else idx.index(x[2]))
    sorted_data_str = "{" + "}, {".join(", ".join(str(i) for i in item) for item in sorted_data) + "}"

    return sorted_data, sorted_data_str

def calculate_mistake_percentage(file1_path, file2_path, ofile1_pth, ofile2_pth):
    def count_words(file_path):
        with open(file_path, 'r', encoding='utf-8') as words_file:
            words_text = words_file.read()
            words_list = words_text.split()
            
        return len(words_list)

    def roll_no(file_path):
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        return file_name
    
    total_words = 0
    full_mistakes = 0
    half_mistakes = 0

    result5, out1, result4 = Splconandtrans(file2_path, file1_path)
    if out1:
        half_mistakes += len(out1)
    
    if result5:
        half_mistakes += len(result5)
    # print(result5)
    if result4:
        half_mistakes += len(result4)

    with open(file2_path, 'r', encoding = 'utf-8') as mfile:
        master_text = mfile.read()
    with open(file1_path, 'r', encoding = 'utf-8') as cfile:
        candidate_text = cfile.read()

    candidate_text = remove_commasandfullstop(candidate_text)
    splitted_ctext = candidate_text.split()
    words_to_replace = []
    for x in out1:
        xlist = x.split(', ')
        splitted_ctext[int(xlist[-2])] = xlist[0][1:]
        
    for x in result4:
        xlist = x.split(', ')
        words = xlist[0].split()
        splitted_ctext[int(xlist[-2])] = words[0][1:]
        splitted_ctext[int(xlist[-2]) + 1] = words[1]
    # print(splitted_ctext)
    # print(missing_splits)
    indexes_to_remove = []
    for x in result5:
        xlist = x.split(', ')
        splitted_ctext[int(xlist[-2])] = xlist[0][1:]
        indexes_to_remove.append(int(xlist[-2])+1)
        
    indexes_to_remove.sort(reverse=True)
    for idx in indexes_to_remove:
        splitted_ctext.pop(idx)
    
        
    candidate_text = ' '.join(splitted_ctext)
    candidate_text = candidate_text.strip()
    candidate_text = candidate_text.lower()
    master_text = remove_commasandfullstop(master_text)
    master_text = master_text.lower()

    a, b, c, z, missed_words, added_words, misspelled, replaced_words = Numberof_mistakes(master_text, candidate_text)

    final_spelling_mistakes = ''
    total_spelling_mistakes = 0 
    for mstk in misspelled:
        final_spelling_mistakes=final_spelling_mistakes+'{'+mstk[0]+', '+mstk[1]+', '+str(mstk[2])+', '+str(mstk[3])+', Spelling Mistake}; '
        total_spelling_mistakes+=1

    errors = []
    total_replaced_words = 0 
    for repl_wrds in replaced_words:
        errors.extend([[repl_wrds[0], repl_wrds[1], repl_wrds[2], repl_wrds[3], 'Replaced Word']])
        total_replaced_words+=1

    #FINAL MISSED WORDS ARE STORED IN BELOW LIST
    total_missed_words_count = 0
    for msd_wrds in missed_words:
        errors.extend([[msd_wrds[0], msd_wrds[1], '99999', 'Missed']])
        total_missed_words_count+=1
        
    #Filter those extra added words which is already taken in spelling mistakes and replace words mistake
    total_count_of_added_words = 0
    for extr_add_wrd in added_words:
        errors.extend([[extr_add_wrd[0], '99999', extr_add_wrd[1], 'Added']])
        total_count_of_added_words+=1
        
    idx = []
    for i in errors:
        # print(i)
        if len(i)==4:
            if 'Missed' in i[3]:
                idx.append(i[1])
            elif 'Added' in i[3]:
                idx.append(i[2])
        else:
            idx.append(i[2])
        idx = sorted(idx)
        
    sorted_data_full = []
    for error in errors:
        if len(error)==4:
            if error[3] == 'Missed':
                sorted_data_full.append((error, idx.index(error[1])))
            elif error[3] == 'Added':
                sorted_data_full.append((error, idx.index(error[2])))
        else:
            sorted_data_full.append((error, idx.index(error[2])))
    
    sorted_data_full.sort(key=lambda x: x[1])
    sorted_data_full = [item[0] for item in sorted_data_full]
    sorted_data_full_err = "{" + "}, {".join(", ".join(str(i) for i in item) for item in sorted_data_full) + "}"


    if replaced_words:
        full_mistakes +=  total_replaced_words
    
    if missed_words:
        full_mistakes += total_missed_words_count

    if added_words:
        full_mistakes += total_count_of_added_words
        
    H_ERROR_CNT = half_mistakes
    F_ERROR_CNT = full_mistakes
    
    if misspelled:
        half_mistakes += total_spelling_mistakes
            
    sorted_data, result8 = find_fullstop_errors(ofile1_pth, ofile2_pth)
    if sorted_data:
        half_mistakes += len(sorted_data)
        

    with open(file2_path, 'r', encoding = 'utf-8') as file2:
        words_file2 = file2.read().lower().split()
        words_file2 = [word.strip(string.punctuation) for word in words_file2]
        total_words = len(words_file2)

    punctuation_set = set(string.punctuation)
    punctuation_count = 0
    tab_count = 0

    with open(file2_path, 'r', encoding = 'utf-8') as file:
        content = file.read()
        for char in content:
            if char in punctuation_set:
                punctuation_count += 1

        for line in file:
            tab_count += line.count('\t')
                    

    total_words_master = total_words + punctuation_count + tab_count
    total_mistakes = (half_mistakes / 2) + full_mistakes 
    mistake_percentage = ((total_mistakes * 100) / 800)
    
    if mistake_percentage > 100:
        mistake_percentage = 100

    path_to_wkhtmltopdf = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe'

    config = pdfkit.configuration(wkhtmltopdf=path_to_wkhtmltopdf)

    # Define your result variables
    missing_space = out1
    transposed_errors = result4
    splitted_words = result5
    misspelled_words = final_spelling_mistakes
    combined_full = sorted_data_full_err
    fullstop_err = result8
    roll_no = roll_no(file1_path)
    words_count1 = count_words(file1_path)
    

    # Function to format error details as a string
    def format_error_details(error_list):
        if len(error_list)>0:
            return ''.join(error_list)
        else:
            return ''

    # Categorize errors
    spelling_mistake = format_error_details(misspelled_words)
    other_than_spelling_mistake = format_error_details(combined_full) 
    
    # Calculate the counts for the other error types
    spacing_cap_transp_mistake = format_error_details(result4) + format_error_details(out1) + format_error_details(splitted_words)

    html_string = f"""
    <!DOCTYPE html>
    <html>
    <head>
    <meta charset="utf-8">
    <style>
        table, th, td {{
            border: 1px solid black;
            border-collapse: collapse;
            font-size: 16px;
            font-family: 'Mangal', sans-serif;  /* Specify Mangal as the font family */
        }}
    </style>
    </head>
    <body  style="padding:15px;">
    <div style="text-align:center;">
    <h2 style="font-size: 32px;">Staff Selection Commission</h2>
    <h2 style="font-size: 32px;">Typing Test Candidate Report</h2>
    </div>

    <div>
    <p style="font-size: 20px;"><b> Name:****** </b></p>
    <p style="font-size: 20px;"><b> Roll Number: {roll_no} </b></p>
    <p style="font-size: 20px;"><b> No. of words typed: {words_count1} </b></p>
    </div>

    <table style="width:100%">
    <tr> 
        <th colspan="2" style="background-color: LightSteelBlue;">Type of Mistakes</th>
        <th style="background-color: LightSteelBlue;">No. of Error</th>
        <th style="background-color: LightSteelBlue;">Error Detail</th>
    </tr>
    <tr>
        <td rowspan="1"><strong> Full Mistake </strong></td>
        <td>Other than Spelling Mistake (Omission/Substitution except transposition/ Addition/ Incomplete Word) </td>
        <td> {(total_replaced_words) + (total_missed_words_count) + (total_count_of_added_words)} </td>
        <td>{other_than_spelling_mistake}</td>
    </tr>
    <tr>
        <td rowspan="3"><strong> Half Mistake </strong></td>
        <td>Splitted/Concatnated/Transposition Mistake</td>
        <td>{len(out1) + len(result4) + len(splitted_words)}</td>
        <td>{spacing_cap_transp_mistake}</td>
    </tr>
    <tr>
        <td>Spelling Mistake</td>
        <td>{total_spelling_mistakes}</td>
        <td>{spelling_mistake}</td>
    </tr>
    <tr>
        <td>Punctuation Error</td>
        <td>{len(sorted_data)}</td>
        <td>{fullstop_err}</td>
    </tr>
    </table>
    </body>
    </html>
    """

    pdf_output_path = r"C:\Users\AnshChoudhary\Downloads\Skilltest-2024\Skilltest-2024\Evaluated\3rd Jan- S1\PDF_Evaluated 3rd Jan- S1\{}.pdf".format(roll_no)

    options = {
        'page-size': 'A4',
        'margin-top': '10mm',
        'margin-right': '30mm',
        'margin-bottom': '10mm',
        'margin-left': '0mm',
        'encoding': 'UTF-8',
    }

    pdfkit.from_string(html_string, pdf_output_path, configuration=config, options=options)

    # print("PDF saved at:", pdf_output_path)
    # print("Total time taken:- ")
    
    return total_spelling_mistakes, H_ERROR_CNT, F_ERROR_CNT, total_mistakes, total_words, mistake_percentage, len(sorted_data)
    
def candidate_hindi_replacement(file1_path):
    with open(file1_path, 'r', encoding='utf-8') as file1:
        candidate_text = file1.read()
    candidate_text = remove_commasandfullstop(candidate_text)
    CwordList = candidate_text.split()
    
    replacements = {
    'अत:': 'अतः',
    'प्रारंभ': 'प्रारम्भ',
    'आरम्भ': 'आरंम्भ', 
    'आरंभ': 'आरंम्भ',
    'किन्तु': 'किंतु',
    'बाँध': 'बांध',
    'उत्तर-प्रदेश': 'उत्तर प्रदेश',
    'चीनी': 'चीनीं',
    'ले जाने': 'ले-जाने',
    'शत-प्रतिशत': 'शतप्रतिशत',
    'सुविधाएँ': 'सुविधाएं',
    'सुविधाऐं': 'सुविधाएं',
    'पन्द्रह': '15', 
    '१५': '15', 
    'पंद्रह': '15',
    '2011': 'दो हजार ग्यारह',
    '२०११': 'दो हजार ग्यारह',
    'पँचायती': 'पंचायती',
    'सँस्थानों': 'संस्थानों',
    '2800': 'दो हजार आठ सौ',
    '२८००': 'दो हजार आठ सौ',
    'पँचायतों': 'पंचायतों',
    'पन्द्रह': '15',
    '१५': '15',
    'खण्डों': 'खंडों',
    'तीन': '3',
    '३': '3',
    'सम्पूर्ण': 'संपूर्ण',
    'जन आंदोलन': 'जन-आंदोलन',
    '2012': 'दो हजार बारह',
    '२०१२': 'दो हजार बारह',
    '1990': 'उन्नीस सौ नब्बे',
    '१९९०': 'उन्नीस सौ नब्बे',
    '2010': 'दो हजार दस',
    '२०१०': 'दो हजार दस',
    'एक दशमलव आठ': '1.8',
    '१ दशमलव ८': '1.8',
    '१.८': '1.8',
    'दो दशमलव पांच': '2.5',
    '२ दशमलव ५': '2.5',
    '२.५': '2.5',
    'दस': '10',
    '१०': '10',
    # 'चार': '4',
    # '४': '4',
    'आँकड़े': 'आंकड़े',
    'सैंतालीस': '47',
    '४७': '47',
    '%': 'प्रतिशत',
    'प्रति शत': 'प्रतिशत',
    'प्रति-शत': 'प्रतिशत',
    'ग्यारह': '11',
    '११': '11',
    'मानदंडों': 'मानदण्डों',
    'गन्दा': 'गंदा',
    'सुविधाऐं': 'सुविधाएं',
    'गाँवों': 'गांवों',
    'गाँव': 'गांव',
    'सड़सठ': '67',
    '६७': '67',
    'छ:': 'छः',
    # '6': 'छः',
    'छह': 'छः',
    '६': 'छः',
    'सम्बधी': 'संबंधी',
    'संबन्धी': 'संबंधी',
    'सम्बन्धी': 'संबंधी',
    'केन्द्रित': 'केंद्रित',
    'आँगनबाड़ियों': 'आंगनबाड़ियों',
    'शत-प्रतिशत': 'शतप्रतिशत',
    'सन्देश': 'संदेश',
    'जाएँगे': 'जाएंगे',
    'उपलब्धियाँ': 'उपलब्धियां',
    'स्तम्भ': 'स्तंभ',
    'चाहूँगी': 'चाहूंगी',
    'ढाँचों': 'ढांचों',
    'अलग अलग': 'अलग-अलग',
    'प्रौद्योगिकियों': 'प्रौद्योगिकियों',
    'निरन्तर': 'निरंतर',
    'ढँग': 'ढंग',
    'पसँद': 'पसंद',
    'हूँ': 'हूं',
    'आन्दोलन': 'आंदोलन',
    'प्रधान मंत्री': 'प्रधानमंत्री',
    'दूँ': 'दूं',
    'मुख्य मंत्री': 'मुख्यमंत्री',
    'स्वतन्त्र': 'स्वतंत्र',
    'स्वत्तंत्रता': 'स्वतंत्रता',
    'सम्भव': 'संभव',
    'सद्भाव': 'सद्भाव',
    'औद्योगिक': 'औद्योगिक',
    '600 करोड़': 'छः सौ करोड़',
    '६०० करोड़': 'छः सौ करोड़',
    '६ सौ करोड़': 'छः सौ करोड़',
    'अन्दर': 'अंदर',
    'बन्द': 'बंद',
    'यहाँ': 'यहां',
    'वहाँ': 'वहां',
    'परन्तु': 'परंतु',
    'साँसदों': 'सांसदों',
    'उद्योग': 'उद्योग',
    'बापूजी': 'बापू जी',
    'गाँधीजी': 'गांधी जी',
    'गांधीजी': 'गांधी जी',
    'गाँधी जी': 'गांधी जी',
    'गाँधी': 'गांधी'
    }

    indexes_to_remove = []
    for idx, word in enumerate(CwordList):
        if word == 'गाँधी' and (idx + 1) < len(CwordList) and CwordList[idx + 1] == 'जी':
            CwordList[idx] = 'गांधी जी'
            indexes_to_remove.append(idx + 1)

    indexes_to_remove.sort(reverse=True)
    for idx in indexes_to_remove:
        CwordList.pop(idx)
    
    for idx, word in enumerate(CwordList):
        if word == '6' and (idx + 1) < len(CwordList) and not(CwordList[idx + 1] == 'सौ' or CwordList[idx + 1] == 'सौं'):
            CwordList[idx] = 'छः'

    for idx, word in enumerate(CwordList):
        if word == 'चार' and (idx + 1) < len(CwordList) and not CwordList[idx + 1] == 'दीवारी':
            CwordList[idx] = '4'

    for idx, word in enumerate(CwordList):
        if word == '४' and (idx + 1) < len(CwordList) and not CwordList[idx + 1] == 'दीवारी':
            CwordList[idx] = '4'

    indexes_to_remove = []
    for idx, word in enumerate(CwordList):
        if word == 'ले' and (idx + 1) < len(CwordList) and CwordList[idx + 1] == 'जाने':
            CwordList[idx] = 'ले-जाने'
            indexes_to_remove.append(idx + 1)

    indexes_to_remove.sort(reverse=True)
    for idx in indexes_to_remove:
        CwordList.pop(idx)
        
    indexes_to_remove = []
    for idx, word in enumerate(CwordList):
        if word == 'अलग' and (idx + 1) < len(CwordList) and CwordList[idx + 1] == 'अलग':
            CwordList[idx] = 'अलग-अलग'
            indexes_to_remove.append(idx + 1)

    indexes_to_remove.sort(reverse=True)
    for idx in indexes_to_remove:
        CwordList.pop(idx)
    
    for idx, word in enumerate(CwordList):
        if word in replacements:
            CwordList[idx] = replacements[word]
            
    indexes_to_remove = []
    for idx, word in enumerate(CwordList):
        if word == '२' and (idx + 1) < len(CwordList) and CwordList[idx + 1] == 'दशमलव' and (idx +2) < len(CwordList) and CwordList[idx + 2] == '५':
            CwordList[idx] = '2.5'
            indexes_to_remove.append(idx + 1)
            indexes_to_remove.append(idx + 2)

    indexes_to_remove.sort(reverse=True)
    for idx in indexes_to_remove:
        CwordList.pop(idx)
    
    indexes_to_remove = []
    for idx, word in enumerate(CwordList):
        if word == 'दो' and (idx + 1) < len(CwordList) and CwordList[idx + 1] == 'दशमलव' and (idx +2) < len(CwordList) and CwordList[idx + 2] == 'पांच':
            CwordList[idx] = '2.5'
            indexes_to_remove.append(idx + 1)
            indexes_to_remove.append(idx + 2)

    indexes_to_remove.sort(reverse=True)
    for idx in indexes_to_remove:
        CwordList.pop(idx)
        
    indexes_to_remove = []
    for idx, word in enumerate(CwordList):
        if word == '१' and (idx + 1) < len(CwordList) and CwordList[idx + 1] == 'दशमलव' and (idx +2) < len(CwordList) and CwordList[idx + 2] == '८':
            CwordList[idx] = '1.8'
            indexes_to_remove.append(idx + 1)
            indexes_to_remove.append(idx + 2)

    indexes_to_remove.sort(reverse=True)
    for idx in indexes_to_remove:
        CwordList.pop(idx)
        
    indexes_to_remove = []
    for idx, word in enumerate(CwordList):
        if word == 'एक' and (idx + 1) < len(CwordList) and CwordList[idx + 1] == 'दशमलव' and (idx +2) < len(CwordList) and CwordList[idx + 2] == 'आठ':
            CwordList[idx] = '1.8'
            indexes_to_remove.append(idx + 1)
            indexes_to_remove.append(idx + 2)

    indexes_to_remove.sort(reverse=True)
    for idx in indexes_to_remove:
        CwordList.pop(idx)
        
    indexes_to_remove = []
    for idx, word in enumerate(CwordList):
        if word == 'जन' and (idx + 1) < len(CwordList) and CwordList[idx + 1] == 'आंदोलन':
            CwordList[idx] = 'जन-आंदोलन'
            indexes_to_remove.append(idx + 1)

    indexes_to_remove.sort(reverse=True)
    for idx in indexes_to_remove:
        CwordList.pop(idx)

    for idx, word in enumerate(CwordList):
        if word in replacements:
            CwordList[idx] = replacements[word]
            
    indexes_to_remove = []
    for idx, word in enumerate(CwordList):
        if word == '600' and (idx + 1) < len(CwordList) and CwordList[idx + 1] == 'करोड़':
            CwordList[idx] = 'छः सौ करोड़'
            indexes_to_remove.append(idx + 1)

    indexes_to_remove.sort(reverse=True)
    for idx in indexes_to_remove:
        CwordList.pop(idx)
        
    indexes_to_remove = []
    for idx, word in enumerate(CwordList):
        if word == '६००' and (idx + 1) < len(CwordList) and CwordList[idx + 1] == 'करोड़':
            CwordList[idx] = 'छः सौ करोड़'
            indexes_to_remove.append(idx + 1)

    indexes_to_remove.sort(reverse=True)
    for idx in indexes_to_remove:
        CwordList.pop(idx)

    indexes_to_remove = []
    for idx, word in enumerate(CwordList):
        if word == '६' and (idx + 1) < len(CwordList) and CwordList[idx + 1] == 'सौ' and (idx +2) < len(CwordList) and CwordList[idx + 2] == 'करोड़':
            CwordList[idx] = 'छः सौ करोड़'
            indexes_to_remove.append(idx + 1)
            indexes_to_remove.append(idx + 2)

    indexes_to_remove.sort(reverse=True)
    for idx in indexes_to_remove:
        CwordList.pop(idx)

    indexes_to_remove = []
    for idx, word in enumerate(CwordList):
        if word == 'छह' and (idx + 1) < len(CwordList) and CwordList[idx + 1] == 'सौ' and (idx +2) < len(CwordList) and CwordList[idx + 2] == 'करोड़':
            CwordList[idx] = 'छः सौ करोड़'
            indexes_to_remove.append(idx + 1)
            indexes_to_remove.append(idx + 2)

    indexes_to_remove.sort(reverse=True)
    for idx in indexes_to_remove:
        CwordList.pop(idx)
        
    indexes_to_remove = []
    for idx, word in enumerate(CwordList):
        if word == 'प्रधान' and (idx + 1) < len(CwordList) and CwordList[idx + 1] == 'मंत्री':
            CwordList[idx] = 'प्रधानमंत्री'
            indexes_to_remove.append(idx + 1)

    indexes_to_remove.sort(reverse=True)
    for idx in indexes_to_remove:
        CwordList.pop(idx)

    indexes_to_remove = []
    for idx, word in enumerate(CwordList):
        if word == 'मुख्य' and (idx + 1) < len(CwordList) and CwordList[idx + 1] == 'मंत्री':
            CwordList[idx] = 'मुख्यमंत्री'
            indexes_to_remove.append(idx + 1)

    indexes_to_remove.sort(reverse=True)
    for idx in indexes_to_remove:
        CwordList.pop(idx)

    Ctext = ' '.join(CwordList)
    Ctext = Ctext.strip()
    # print(Ctext)

    return Ctext

def main(main_pth):
    start_time = time.time()
    cols = ['asr_rollno', 'asr_region_code', 'asr_date_appeared', 'asr_batch_no', 'asr_spelling_mistakes', 'asr_half_error', 'asr_misc_error', 'asr_punctuation_error','asr_total_mistakes', 'asr_no_of_word_original', 'asr_per_of_error', 'asr_stenograde', 'asr_remarks']
    ldc_error = pd.DataFrame(columns= cols)

    main_folder = os.listdir(main_pth)
    file1_pth = ""
    file2_pth = ""
    allowed_spellings_pth = ""
    asr_remarks = "Steno-Hindi"
    asr_stenograde = "Grade-D"
    temp_root = r"C:\Users\AnshChoudhary\Downloads\Skilltest-2024\Skilltest-2024\Evaluated\3rd Jan- S1\temp_text"
    
    for root, dirs, files in os.walk(main_pth):
        for file in files:
            if file.endswith(".csv"):
                allowed_spellings_pth = os.path.join(root, file)  
                #print(allowed_spellings_pth)  

            elif file.endswith(".txt"):
                if file.lower().startswith("master"):
                    ofile2_pth = os.path.join(root, file)

                    temp_file2 = candidate_hindi_replacement(ofile2_pth)
                    temp_pth = os.path.join(temp_root,"master.txt") 
                    with open(temp_pth, 'w', encoding='utf-8') as file2:
                        file2.write(temp_file2)
                    
                    file2_pth = temp_pth

                else:
                    ofile1_pth = os.path.join(root, file)
                    
                    path_lst = ofile1_pth.split('\\')
                    f = path_lst[-1].split('.')
                    asr_rollno = f[-2]
                    first_digit = str(asr_rollno)[0]
                    if first_digit == '1':
                        asr_region_code = 'NWR'
                    elif first_digit == '2':
                        asr_region_code = 'NR'
                    elif first_digit == '3':
                        asr_region_code = 'CR'
                    elif first_digit == '4':
                        asr_region_code = 'ER'
                    elif first_digit == '5':
                        asr_region_code = 'NCR'
                    elif first_digit == '6':
                        asr_region_code = 'MPR'
                    elif first_digit == '7':
                        asr_region_code = 'WR'
                    elif first_digit == '8':
                        asr_region_code = 'SR'
                    elif first_digit == '9':
                        asr_region_code = 'KKR'
                        
                    asr_date_appeared = '03-01-2024'
                    asr_batch_no =  'Shift-2'

                    temp_file1 = candidate_hindi_replacement(ofile1_pth)
                    temp_pth = os.path.join(temp_root,f"{asr_rollno}.txt") 
                    with open(temp_pth, 'w', encoding = 'utf-8') as file1:
                        file1.write(temp_file1)

                    file1_pth = temp_pth

                    total_spelling_mistakes, H_ERROR_CNT, F_ERROR_CNT, total_mistakes, total_words, mistake_percentage,result8 = calculate_mistake_percentage(file1_pth, file2_pth, ofile1_pth, ofile2_pth) 
                    
                    row = {'asr_rollno': asr_rollno,
                           'asr_region_code': asr_region_code,
                           'asr_date_appeared': asr_date_appeared,
                           'asr_batch_no': asr_batch_no,
                           'asr_remarks': asr_remarks,
                           'asr_stenograde': asr_stenograde,
                           'asr_spelling_mistakes': total_spelling_mistakes,
                           'asr_half_error': H_ERROR_CNT,
                           'asr_misc_error': F_ERROR_CNT,
                           'asr_punctuation_error': result8,
                           'asr_total_mistakes': total_mistakes,
                           'asr_no_of_word_original': 800,
                           'asr_per_of_error': mistake_percentage}

                    ldc_error = pd.concat([ldc_error, pd.DataFrame([row])], ignore_index=True)

    output_excel_path = r"C:\Users\AnshChoudhary\Downloads\Skilltest-2024\Skilltest-2024\Evaluation\3rd Jan- S1\Percentage_report 3rd Jan- S1.xlsx"
    
    end_time = time.time()
    
    total_time = end_time - start_time
    
    
    print(total_time)
    ldc_error.to_excel(output_excel_path, index=False)

main(r"C:\Users\AnshChoudhary\Downloads\Skilltest-2024\Skilltest-2024\Evaluation\3rd Jan- S1") 






C:\Users\AnshChoudhary\AppData\Local\Temp\ipykernel_12296\1915974265.py:1349: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ldc_error = pd.concat([ldc_error, pd.DataFrame([row])], ignore_index=True)


5234.635542392731
